# Growthcurves annotate_plot tutorial

This notebook focuses on `growthcurves.plot.annotate_plot()`, which adds
annotations to a Plotly figure only when the corresponding inputs are provided.
Pass `None` (or omit a parameter) to hide that annotation.

In [1]:
import growthcurves as gc
import pandas as pd
import numpy as np

# Example data
data = [
    0.0493,
    0.0494,
    0.0492,
    0.0491,
    0.0491,
    0.0492,
    0.0492,
    0.0490,
    0.0491,
    0.0491,
    0.0492,
    0.0490,
    0.0492,
    0.0490,
    0.0490,
    0.0489,
    0.0491,
    0.0491,
    0.0491,
    0.0491,
    0.0491,
    0.0491,
    0.0492,
    0.0491,
    0.0492,
    0.0492,
    0.0492,
    0.0493,
    0.0492,
    0.0493,
    0.0495,
    0.0494,
    0.0494,
    0.0494,
    0.0494,
    0.0494,
    0.0495,
    0.0494,
    0.0495,
    0.0497,
    0.0496,
    0.0496,
    0.0498,
    0.0497,
    0.0498,
    0.0498,
    0.0498,
    0.0499,
    0.0500,
    0.0499,
    0.0501,
    0.0499,
    0.0501,
    0.0500,
    0.0499,
    0.0502,
    0.0502,
    0.0503,
    0.0502,
    0.0503,
    0.0503,
    0.0503,
    0.0504,
    0.0503,
    0.0504,
    0.0506,
    0.0506,
    0.0506,
    0.0506,
    0.0507,
    0.0508,
    0.0509,
    0.0509,
    0.0509,
    0.0510,
    0.0510,
    0.0510,
    0.0512,
    0.0512,
    0.0513,
    0.0514,
    0.0513,
    0.0514,
    0.0515,
    0.0515,
    0.0516,
    0.0515,
    0.0518,
    0.0518,
    0.0518,
    0.0520,
    0.0519,
    0.0520,
    0.0522,
    0.0521,
    0.0522,
    0.0523,
    0.0523,
    0.0525,
    0.0526,
    0.0527,
    0.0528,
    0.0528,
    0.0527,
    0.0530,
    0.0531,
    0.0531,
    0.0532,
    0.0534,
    0.0533,
    0.0535,
    0.0534,
    0.0537,
    0.0537,
    0.0540,
    0.0539,
    0.0540,
    0.0540,
    0.0543,
    0.0544,
    0.0545,
    0.0546,
    0.0547,
    0.0549,
    0.0549,
    0.0551,
    0.0552,
    0.0553,
    0.0554,
    0.0555,
    0.0557,
    0.0557,
    0.0559,
    0.0560,
    0.0560,
    0.0564,
    0.0565,
    0.0567,
    0.0567,
    0.0570,
    0.0571,
    0.0574,
    0.0575,
    0.0576,
    0.0579,
    0.0581,
    0.0582,
    0.0583,
    0.0585,
    0.0588,
    0.0591,
    0.0592,
    0.0594,
    0.0598,
    0.0600,
    0.0602,
    0.0606,
    0.0610,
    0.0613,
    0.0616,
    0.0620,
    0.0624,
    0.0627,
    0.0633,
    0.0638,
    0.0643,
    0.0649,
    0.0655,
    0.0659,
    0.0666,
    0.0672,
    0.0679,
    0.0685,
    0.0695,
    0.0703,
    0.0712,
    0.0722,
    0.0732,
    0.0741,
    0.0754,
    0.0766,
    0.0780,
    0.0792,
    0.0806,
    0.0821,
    0.0834,
    0.0853,
    0.0867,
    0.0885,
    0.0900,
    0.0920,
    0.0941,
    0.0962,
    0.0982,
    0.1002,
    0.1024,
    0.1044,
    0.1063,
    0.1078,
    0.1100,
    0.1120,
    0.1141,
    0.1163,
    0.1188,
    0.1214,
    0.1238,
    0.1262,
    0.1290,
    0.1319,
    0.1346,
    0.1376,
    0.1407,
    0.1438,
    0.1468,
    0.1502,
    0.1538,
    0.1576,
    0.1615,
    0.1653,
    0.1694,
    0.1735,
    0.1783,
    0.1834,
    0.1895,
    0.1949,
    0.2008,
    0.2072,
    0.2128,
    0.2194,
    0.2255,
    0.2321,
    0.2386,
    0.2451,
    0.2523,
    0.2590,
    0.2647,
    0.2700,
    0.2752,
    0.2812,
    0.2866,
    0.2920,
    0.2969,
    0.3022,
    0.3081,
    0.3130,
    0.3187,
    0.3233,
    0.3280,
    0.3328,
    0.3363,
    0.3409,
    0.3451,
    0.3486,
    0.3532,
    0.3570,
    0.3599,
    0.3634,
    0.3670,
    0.3703,
    0.3736,
    0.3773,
    0.3806,
    0.3845,
    0.3882,
    0.3933,
    0.3992,
    0.4032,
    0.4068,
    0.4108,
    0.4146,
    0.4177,
    0.4201,
    0.4236,
    0.4263,
    0.4288,
    0.4322,
    0.4347,
    0.4373,
    0.4394,
    0.4412,
    0.4426,
    0.4440,
    0.4461,
    0.4481,
    0.4487,
    0.4500,
    0.4514,
    0.4522,
    0.4534,
    0.4532,
    0.4541,
    0.4545,
    0.4552,
    0.4553,
    0.4557,
    0.4566,
    0.4561,
    0.4571,
    0.4579,
    0.4579,
    0.4588,
    0.4587,
    0.4600,
    0.4595,
    0.4601,
    0.4596,
    0.4597,
    0.4601,
    0.4603,
    0.4598,
    0.4596,
    0.4597,
    0.4595,
    0.4600,
    0.4607,
    0.4606,
    0.4606,
    0.4601,
    0.4605,
    0.4602,
    0.4650,
    0.4646,
    0.4637,
    0.4622,
    0.4609,
    0.4605,
    0.4601,
    0.4597,
    0.4605,
    0.4600,
    0.4598,
    0.4600,
    0.4604,
    0.4602,
    0.4605,
    0.4606,
    0.4611,
    0.4604,
    0.4605,
    0.4609,
    0.4604,
    0.4615,
    0.4613,
    0.4615,
    0.4615,
    0.4616,
    0.4616,
    0.4617,
    0.4622,
    0.4624,
    0.4621,
    0.4621,
    0.4625,
    0.4626,
    0.4625,
    0.4626,
    0.4627,
    0.4623,
    0.4628,
    0.4628,
    0.4625,
    0.4633,
    0.4632,
    0.4632,
    0.4641,
    0.4644,
    0.4645,
    0.4645,
    0.4646,
    0.4652,
    0.4661,
    0.4659,
    0.4662,
    0.4666,
    0.4661,
    0.4673,
    0.4675,
    0.4678,
    0.4674,
    0.4683,
    0.4683,
    0.4684,
    0.4693,
    0.4697,
    0.4697,
    0.4702,
    0.4699,
    0.4701,
    0.4715,
    0.4716,
    0.4718,
    0.4723,
    0.4724,
    0.4724,
    0.4734,
    0.4730,
    0.4734,
    0.4744,
    0.4738,
    0.4745,
    0.4750,
    0.4758,
    0.4762,
    0.4764,
    0.4766,
    0.4771,
    0.4775,
    0.4777,
    0.4784,
    0.4783,
    0.4787,
    0.4796,
    0.4800,
    0.4805,
    0.4812,
    0.4817,
    0.4825,
    0.4826,
    0.4826,
    0.4833,
    0.4837,
    0.4843,
    0.4842,
    0.4844,
    0.4851,
    0.4860,
    0.4867,
    0.4864,
    0.4872,
    0.4876,
    0.4880,
    0.4889,
    0.4896,
    0.4893,
    0.4901,
    0.4906,
    0.4907,
    0.4913,
    0.4920,
    0.4926,
    0.4929,
    0.4926,
    0.4932,
    0.4940,
    0.4943,
    0.4949,
    0.4955,
    0.4957,
    0.4960,
    0.4968,
    0.4972,
    0.4982,
    0.4982,
    0.4988,
    0.4989,
    0.4999,
    0.5000,
    0.5006,
    0.5012,
    0.5013,
    0.5016,
    0.5017,
    0.5027,
    0.5023,
    0.5034,
    0.5034,
    0.5043,
    0.5039,
    0.5055,
    0.5053,
    0.5064,
    0.5065,
    0.5067,
    0.5070,
    0.5075,
    0.5083,
    0.5090,
    0.5091,
    0.5098,
    0.5097,
    0.5100,
    0.5109,
    0.5103,
    0.5110,
    0.5122,
    0.5123,
    0.5131,
    0.5129,
    0.5130,
    0.5139,
    0.5140,
    0.5144,
    0.5157,
    0.5158,
    0.5162,
    0.5172,
    0.5171,
    0.5170,
    0.5174,
    0.5190,
    0.5186,
    0.5193,
    0.5195,
    0.5203,
    0.5202,
    0.5211,
    0.5212,
    0.5224,
    0.5224,
    0.5227,
    0.5229,
    0.5247,
    0.5246,
    0.5251,
    0.5255,
    0.5262,
    0.5270,
    0.5273,
    0.5273,
    0.5279,
    0.5289,
    0.5290,
    0.5288,
    0.5298,
    0.5305,
    0.5309,
    0.5309,
    0.5319,
    0.5322,
    0.5325,
    0.5331,
    0.5337,
    0.5342,
    0.5344,
    0.5347,
    0.5359,
    0.5364,
    0.5361,
    0.5372,
    0.5372,
    0.5378,
    0.5381,
    0.5383,
    0.5389,
    0.5396,
    0.5401,
    0.5405,
    0.5400,
    0.5408,
    0.5417,
    0.5424,
    0.5447,
    0.5473,
    0.5485,
    0.5501,
    0.5499,
    0.5494,
    0.5493,
    0.5489,
    0.5486,
    0.5480,
    0.5475,
    0.5472,
    0.5475,
    0.5471,
    0.5464,
    0.5462,
    0.5457,
    0.5457,
    0.5452,
    0.5456,
    0.5455,
    0.5453,
    0.5445,
    0.5451,
    0.5457,
    0.5449,
    0.5447,
    0.5448,
    0.5453,
    0.5452,
    0.5450,
    0.5444,
    0.5444,
    0.5450,
    0.5451,
    0.5452,
    0.5444,
    0.5454,
    0.5455,
    0.5454,
    0.5456,
    0.5461,
    0.5466,
    0.5467,
    0.5468,
    0.5473,
    0.5471,
    0.5478,
    0.5476,
    0.5476,
    0.5483,
    0.5486,
    0.5493,
    0.5485,
    0.5488,
    0.5490,
    0.5503,
    0.5496,
    0.5497,
    0.5502,
    0.5502,
    0.5503,
    0.5500,
    0.5505,
    0.5504,
    0.5512,
    0.5514,
    0.5507,
    0.5512,
    0.5513,
    0.5517,
    0.5519,
    0.5519,
    0.5517,
    0.5523,
    0.5526,
    0.5522,
    0.5518,
    0.5527,
    0.5522,
    0.5525,
    0.5527,
    0.5529,
    0.5533,
    0.5532,
    0.5537,
    0.5533,
    0.5538,
    0.5540,
    0.5538,
    0.5538,
    0.5541,
    0.5541,
    0.5541,
    0.5545,
    0.5541,
    0.5550,
    0.5549,
    0.5555,
    0.5552,
    0.5552,
    0.5556,
    0.5554,
    0.5552,
    0.5560,
    0.5561,
    0.5558,
    0.5564,
    0.5567,
    0.5559,
    0.5567,
    0.5566,
    0.5566,
    0.5567,
    0.5560,
    0.5571,
    0.5568,
    0.5572,
    0.5569,
    0.5576,
    0.5580,
    0.5578,
    0.5575,
    0.5581,
    0.5583,
    0.5577,
    0.5578,
    0.5581,
    0.5586,
    0.5590,
    0.5586,
    0.5594,
    0.5598,
    0.5591,
    0.5597,
    0.5598,
    0.5602,
    0.5601,
    0.5604,
    0.5602,
    0.5603,
    0.5606,
    0.5608,
]

time = np.array([(12 * n) / 60 for n in range(len(data))])

## Compare growth statistics across methods

This section computes growth statistics using multiple fitting methods and
compares them in a grid of bar plots.

### Note on Fit Results Structure

All fitting functions (`fit_model` and `fit_non_parametric`) return a dictionary with:
- `model_type`: The type of model or method used
- `params`: A dictionary containing all model parameters, **including `fit_t_min` and `fit_t_max`** which indicate the time window used for fitting

This structure ensures all fitting outputs are consistently organized.

In [ ]:
# fit parametric models
logistic_result = gc.parametric.fit_parametric(time, data, model_type="logistic")
gompertz_result = gc.parametric.fit_parametric(time, data, model_type="gompertz")
richards_result = gc.parametric.fit_parametric(time, data, model_type="richards")

# fit non-parametric models
spline_result = gc.non_parametric.fit_non_parametric(time, data, umax_method="spline")
window_result = gc.non_parametric.fit_non_parametric(
    time, data, umax_method="sliding_window"
)

# extract stats from fits
logistic_stats = gc.utils.extract_stats_from_fit(logistic_result, time, data)
gompertz_stats = gc.utils.extract_stats_from_fit(gompertz_result, time, data)
richards_stats = gc.utils.extract_stats_from_fit(richards_result, time, data)
spline_stats = gc.utils.extract_stats_from_fit(spline_result, time, data)
window_stats = gc.utils.extract_stats_from_fit(window_result, time, data)

methods = {
    "Logistic": logistic_stats,
    "Gompertz": gompertz_stats,
    "Richards": richards_stats,
    "Spline": spline_stats,
    "Sliding window": window_stats,
}

# Use the output keys as the metrics list (same across methods)
metrics = [k for k in next(iter(methods.values())).keys() if k != "fit_method"]

# Display stats in a pandas table
stats_df = pd.DataFrame(methods).T
stats_df = stats_df[metrics]
stats_df

,max_od,specific_growth_rate,doubling_time,exp_phase_start,exp_phase_end,time_at_umax,od_at_umax,fit_t_min,fit_t_max,model_rmse
Logistic,0.517948,0.100234,6.915302,31.108617,65.828056,41.663327,0.152112,0.0,138.6,0.025333
Gompertz,0.524529,0.129449,5.35459,32.497395,69.716633,38.608016,0.106602,0.0,138.6,0.022813
Richards,0.524438,0.128803,5.381446,32.497395,69.716633,38.608016,0.106711,0.0,138.6,0.022841
Spline,0.5608,0.116602,5.944552,35.789318,58.616083,42.903518,0.152063,35.8,58.6,0.027819
Sliding window,0.5608,0.144226,4.805964,35.789318,58.616083,45.2,0.2072,43.8,46.6,0.002061


## Access fitted parameters

Each fit result includes a `params` dictionary with the fitted model parameters.
Below are examples for both parametric and non-parametric fits.

In [10]:
# Access fitted parameters from each model
# Note: fit_t_min and fit_t_max are now included in the params dictionary
from pprint import pprint

parametric_params = {
    "Logistic": logistic_result,
    "Gompertz": gompertz_result,
    "Richards": richards_result,
}

non_parametric_params = {
    "Spline": spline_result,
    "Sliding window": window_result,
}

print("=== Parametric Model Parameters ===")
pprint(parametric_params)
print("\n=== Non-Parametric Model Parameters ===")
pprint(non_parametric_params)

=== Parametric Model Parameters ===
{'Gompertz': {'model_type': 'gompertz',
              'params': {'K': np.float64(0.5245286195029899),
                         'fit_t_max': 138.6,
                         'fit_t_min': 0.0,
                         'lam': np.float64(36.5530438012088),
                         'mu_max_param': np.float64(0.020172534003075532),
                         'y0': np.float64(0.05103740537395796)}},
 'Logistic': {'model_type': 'logistic',
              'params': {'K': np.float64(0.5179477199372282),
                         'fit_t_max': 138.6,
                         'fit_t_min': 0.0,
                         'r': np.float64(0.184455480980883),
                         't0': np.float64(48.346737932212086),
                         'y0': np.float64(0.0454790298563513)}},
 'Richards': {'model_type': 'richards',
              'params': {'K': np.float64(0.524437849172164),
                         'fit_t_max': 138.6,
                         'fit_t_min': 0.0,
   

In [5]:
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Build plots from the pandas table
metrics = list(stats_df.columns)
n_metrics = len(metrics)
n_cols = 3
n_rows = math.ceil(n_metrics / n_cols)

fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    subplot_titles=[key.replace("_", " ").title() for key in metrics],
    horizontal_spacing=0.08,
    vertical_spacing=0.15,
)

method_names = list(stats_df.index)
for i, key in enumerate(metrics):
    row = i // n_cols + 1
    col = i % n_cols + 1
    values = stats_df[key]
    numeric_values = values.apply(lambda v: pd.to_numeric(v, errors="coerce"))
    has_numeric = numeric_values.notna().any()

    if has_numeric:
        fig.add_trace(
            go.Bar(
                x=method_names,
                y=numeric_values.tolist(),
                showlegend=False,
                marker=dict(line=dict(color="black", width=1)),
            ),
            row=row,
            col=col,
        )
        fig.update_yaxes(
            title_text=key.replace("_", " ").title(), automargin=True, row=row, col=col
        )
    else:
        fig.add_trace(
            go.Bar(
                x=method_names,
                y=[0] * len(method_names),
                showlegend=False,
                marker=dict(line=dict(color="black", width=1)),
            ),
            row=row,
            col=col,
        )
        fig.update_yaxes(showticklabels=False, automargin=True, row=row, col=col)

    fig.update_xaxes(tickangle=20, automargin=True, row=row, col=col)

fig.update_layout(
    title="Growth statistics comparison across methods",
    height=300 * n_rows,
    width=1100,
    margin=dict(l=80, r=30, t=80, b=80),
)

fig.show()

## Plotting API

You can now pass fit results directly to `annotate_plot()` without manually constructing a fitted model dictionary. The function automatically extracts `fit_t_min` and `fit_t_max` from the params to determine the fitting window.

Note: The plotting functions no longer show titles or legends by default. `time_umax` adds a vertical line, and `od_umax` adds a horizontal line at the respective values.

In [ ]:
# Fit a non-parametric model and extract statistics
spline_result = gc.non_parametric.fit_non_parametric(time, data, umax_method="spline")
spline_stats = gc.utils.extract_stats_from_fit(spline_result, time, data)

# Plot with annotations (spline)
# Simply pass the fit result directly to annotate_plot
# Scale is automatically detected from the figure
fig = gc.plot.create_base_plot(time, data, scale="linear")
fig = gc.plot.annotate_plot(
    fig,
    phase_boundaries=(spline_stats["exp_phase_start"], spline_stats["exp_phase_end"]),
    time_umax=spline_stats["time_at_umax"],
    od_umax=spline_stats["od_at_umax"],
    od_max=spline_stats["max_od"],
    umax_point=(spline_stats["time_at_umax"], spline_stats["od_at_umax"]),
    fitted_model=spline_result,  # Pass fit result directly!
)

fig.show()

## Annotate plot with Gompertz fit

Same annotation workflow using a parametric Gompertz fit. Note that `time_umax` adds a vertical line and `od_umax` adds a horizontal line at the OD value at maximum growth rate.

In [ ]:
# Fit a parametric model and extract statistics
gompertz_result = gc.parametric.fit_parametric(time, data, model_type="gompertz")
gompertz_stats = gc.utils.extract_stats_from_fit(gompertz_result, time, data)

# Example: Plot in log scale
# Scale is automatically detected - no need to specify in annotate_plot
fig_log = gc.plot.create_base_plot(time, data, scale="log")
fig_log = gc.plot.annotate_plot(
    fig_log,
    phase_boundaries=(
        gompertz_stats["exp_phase_start"],
        gompertz_stats["exp_phase_end"],
    ),
    time_umax=gompertz_stats["time_at_umax"],
    od_umax=gompertz_stats["od_at_umax"],
    od_max=gompertz_stats["max_od"],
    umax_point=(gompertz_stats["time_at_umax"], gompertz_stats["od_at_umax"]),
    fitted_model=gompertz_result,
)

fig_log.show()